# Introduction for Developers

This notebook presents a minimal example of how to implement a neural network and and an application with deeplay.
Specifically, it implements the classes for a multilayer perceptron and a classifier.
Then, it combines them to demonstrate how tehy can be used for a simple classification task.
Finally, it upgrades these classes adding functionalities that are required to improved the user experience when using an IDE.

## Minimal Multilayer Perceptron

Here, we implement the minimal class `SimpleMLP`. It extends directly `dl.DeeplayModule`, which is the base class for all modules in `deeplay`.

It represents a multilayer perceptron with a certain umber of inputs (`ìn_features`, which is an integer), a series of hidden layers with a certain number of neurons (`hidden_features`, a vector with the number of neurons for each layer), and a certain numebr of outputs (`out_features`, which is an integer).

The constructor initializes the MLP by creating a sequence of linear and ReLU activation layers.

The `forward` method defines the data flow through the network, sequentially passing the input through each linear-activation block and returning the final output.

In [ ]:
import deeplay as dl
import torch.nn as nn

class SimpleMLP(dl.DeeplayModule):

    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        
        self.blocks = dl.LayerList()
        for inputs_layer, outputs_layer in zip([in_features, *hidden_features], 
                                               [*hidden_features, out_features]):
            self.blocks.append(
                dl.LayerActivationBlock(
                    dl.Layer(nn.Linear, inputs_layer, outputs_layer),
                    dl.Layer(nn.ReLU)
                )
            )

    def forward(self, x):
        for block in self.blocks:
            x = block(x)

        return x

We can now create an instance of `SimpleMLP` in various ways, for example:

```python
mlp = SimpleMLP(2, [32, 32], 2)
```

or more explicitly:

```python
mlp = SimpleMLP(
    in_features=2, 
    hidden_feature=[32, 32], 
    out_features=2
)
```

In [ ]:
mlp = SimpleMLP(2, [32, 32], 2)

print(mlp)

## Minimal Classifier

Here, we now implement the application `SimpleClassifier`. This extend the deeplay class `dl.Application`.

In [ ]:
class SimpleClassifier(dl.Application):

    def __init__(self, model, **kwargs):
        self.model = model
        super().__init__(**kwargs)
        
        
    def forward(self, x):
        return self.model(x)

We can now create an instance of this using as model the `mlp` that we have defined above, setting `loss` to `nn.CrossEntropyLoss()` and `optimizer`to `dl.Adam(lr=1e-3)`. We also add kepp track of a metrics, setting `metrics` to `[tm.Accuracy("multiclass", num_classes=2)]`.

Since we are using a cross-entropy loss, we need to set the output activation to `nn.Identity`.

In [ ]:
import torchmetrics as tm

classifier = SimpleClassifier(
    model=mlp, 
    loss=nn.CrossEntropyLoss(), 
    optimizer=dl.Adam(lr=1e-3), 
    metrics=[tm.Accuracy("multiclass", num_classes=2)]
)
classifier.model.blocks[-1].activation.configure(nn.Identity)

classifier.build()

print(classifier)

**Notes**

Instead of `classifier.build()`, which build the module in place, it is also possible to use `new_classifier = classifier.create()`, which clones and build the classifier.

Instead of `classifier.model.blocks[-1].activation.configure(nn.Identity)`, it'd also be possible to use `classifier.model.blocks[-1].activation.configure(nn.Identity)`, which is more easily understandable.

## Example

We'll now use `classifier` for the simple task of determinig whether the sum of two numbers is larger or smaller than 0.

In [ ]:
from torch import randn
from torch.utils.data import TensorDataset, random_split, DataLoader

num_samples = 100
data = randn(num_samples, 2)
labels = (data.sum(dim=1) > 0).long()

dataset = TensorDataset(data, labels)
train, val = random_split(dataset, [0.8, 0.2])

train_dataloader = DataLoader(train, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val, batch_size=16, shuffle=False)

trainer = dl.Trainer(max_epochs=10)

trainer.fit(classifier, train_dataloader, val_dataloader)

trainer.test(classifier, val_dataloader)

## Quality-of-Life Improvements for IDE